In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
json_data = pd.read_json('data/world_bank_projects.json')
json_data.head(2)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


In [3]:
json_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

In [4]:
json_data.groupby('countryname').size().sort_values(ascending=False).head(10)


countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
dtype: int64

In [5]:
print(json_data['mjtheme_namecode'][0])
dict = json_data['mjtheme_namecode']

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]


In [6]:
#json_data['mjtheme_namecode']
#json_data.groupby('mjtheme_namecode').size()
#column = json_data['mjtheme_namecode']
#sector = pd.DataFrame(json_data['mjtheme_namecode'])
#type(sector)
#projects_count = []
#for column['name'] in sector:
 #   projects_count.append(column['name'])
    
                         
#print(projects_count)
from collections import OrderedDict

dict = OrderedDict(dict)
#print(dict['name'])
for name in dict.values():
    print(name)

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
[{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]
[{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]
[{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]
[{'code': '5', 'name': 'Trade and integration'}, {'code': '4', 'name': 'Financial and private sector development'}]
[{'code': '6', 'name': 'Social protection and risk management'}, {'code': '6', 'name': ''}]
[{'code': '2', 'name': 'Public sector governance'}, {'code': '4', 'name': 'Financial and private sector development'}]
[{'code': '11', 'name': 'Environment and natural resources management'}, {'code': '8', 'name': ''}]
[{'code': '10', 'name': 'Rural development'}

In [15]:

# countries with most projects
json_data.groupby('countryshortname').size().nlargest(10)

countryshortname
China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Morocco               12
Nepal                 12
Africa                11
Mozambique            11
dtype: int64

In [14]:

# using project themes
with open('data/world_bank_projects.json') as json_dt:
    my_data = json.load(json_dt)

df_themes = json_normalize(my_data, 'mjtheme_namecode', ['id'])

df_themes.groupby('id').code.nunique().sort_values(ascending=False).head(10)

df_themes.name.value_counts().head(10)


Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

In [11]:
# dealing with missing values
import numpy as np
the_themes = df_themes.sort_values('code')
the_themes['name'] = the_themes.name.replace('', method='pad')
the_themes = the_themes.groupby('name', as_index=False)['name'].agg({'total_number' : np.size})
the_themes = the_themes.rename(columns={'name': 'project_theme'}) 
the_themes.sort_values(by='total_number', ascending=False).head(10)


,project_theme,total_number
1,Environment and natural resources management,250
6,Rural development,216
3,Human development,210
4,Public sector governance,199
8,Social protection and risk management,168
2,Financial and private sector development,146
7,Social dev/gender/inclusion,130
9,Trade and integration,77
10,Urban development,50
0,Economic management,38
